# Get Data

Note: data is the three scale channel features

In [ ]:
import gc
import h5py
import numpy as np

# load_path='F:/wangqi/nistdataset/AAPlantD1_2GHz_TX1_hpol_run4_pp.mat'  # 3600:39600 120
# load_mat=h5py.File(load_path,'r')
# IQdata=load_mat['IQdata'][3600:39600,0:256].view('complex')
# locations=120
# records=300

load_path='F:/wangqi/nistdataset/AAPlantD1_2GHz_TX1_vpol_run3_pp.mat'   # 6600:42600  120
load_mat=h5py.File(load_path,'r')
IQdata=load_mat['IQdata'][6600:42600,0:256].view('complex')
locations=120
records=300

# load_path='F:/wangqi/nistdataset/AAPlantD3_2GHz_TX2b_vpol_run32_pp.mat'  # 3900:39900 120
# load_mat=h5py.File(load_path,'r')
# IQdata=load_mat['IQdata'][3900:39900,0:256].view('complex')
# locations=120
# records=300

# load_path='F:/wangqi/nistdataset/AAPlantD3_2GHz_TX2b_hpol_run34_pp.mat'  # 3900:39900 120
# load_mat=h5py.File(load_path,'r')
# IQdata=load_mat['IQdata'][2100:33900,0:256].view('complex')
# locations=106
# records=300

# Hyper-parameters

In [ ]:
# training parameters
seed = 0                        # random seed
batch_size = 32                # batch size
num_epoch = 10                  # the number of training epoch
patience = 10                  # If no improvement in 'patience' epochs, early stop
learning_rate = 0.0001         # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
k=1  #H为k+1个h
nf=2 #参考CSI的数量+1
index=[0,1,2]
# index=[0,1,3,2]
# index=[0,1,3,2,4]
# index=[0,1,4,2,5,3,6,7]
# index=[0,1,4,2,5,3,6]
d=2  #eve在alice后面d-1个位置处尾随
input_channels=4
len_sequence = 256                  # the input dim of the model, you should not change the value
hidden_layers = 1               # the number of hidden layers
hidden_dim = 64                # the hidden dim
output_dim=2
mu=0.4
std=0.01

# creat a image

In [ ]:
import numpy as np
from numpy import fft
def convert_pic(h):
    width, height=h.shape  # width=128,height=k+nf
    img=np.zeros([input_channels,len_sequence,height-1]).astype(np.float32)
    for i in range(1,height):
        img[0,:,i-1]=np.abs(h[:,i]*np.conj(h[:,0]))
        img[1,:,i-1]=np.angle(h[:,i]*np.conj(h[:,0]))
    hf=fft.fft2(h)
    for i in range(1,height):
        img[2,:,i-1]=np.abs(hf[:,i]*np.conj(hf[:,0]))
        img[3,:,i-1]=np.angle(hf[:,i]*np.conj(hf[:,0]))
    return img

# creatH

In [ ]:
def creatHimag_Alice(idx, k, nf):
    H=np.zeros((k+nf,len_sequence)).astype(complex)
    H=IQdata[idx-nf:idx+k]
    # print(np.shape(H))
    return  np.transpose(H[index])
def creatHimag_Eve(idx, k, nf, d):
    H=np.zeros((k+nf,len_sequence)).astype(complex)
    H[nf:k+nf,:]=IQdata[idx-d*records:idx+k-d*records]
    H[0:nf,:]=IQdata[idx-nf:idx]
    return  np.transpose(H[index])

# Preprocess data 

In [ ]:
recs=records*d
num_dataall=locations*records-recs-10
dataall_HimagA=np.zeros((num_dataall,len_sequence, k+nf)).astype(complex)
dataall_HimagE=np.zeros((num_dataall,len_sequence, k+nf)).astype(complex)
for i in range(recs,locations*records-10):
    dataall_HimagA[i-recs,:,:]=creatHimag_Alice(i,k=k,nf=nf)
    dataall_HimagE[i-recs,:,:]=creatHimag_Eve(i,k=k,nf=nf,d=d)

dataall_TW=np.concatenate((dataall_HimagA,dataall_HimagE),axis=0)
location=[i for i in range(2)]
test_data=dataall_TW
test_lab=[val for val in location for i in range(int(num_dataall))]

del dataall_HimagA, dataall_HimagE, dataall_TW
gc.collect()

# Define Dataset

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class hDataset(Dataset):
    def __init__(self, k, X, y=None):
        super(hDataset).__init__()
        # self.data = torch.DoubleTensor(abs(X))
        self.data = X
        self.k = k
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        h = self.data[idx]
        h = convert_pic(h)
        if self.label is not None:
            return h, self.label[idx]
        else:
            return h

    def __len__(self):
        return len(self.data)


# Testing

In [ ]:
import torch
from model import resnet34

device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

import numpy as np
test_set = hDataset(k, test_data, test_lab)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

_exp_name="Corrm9947_run4_resnettwi_sample"
model_best = resnet34(num_classes=2).to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt",map_location='cpu'))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in test_loader:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

accuracy=1 - np.count_nonzero(np.array(prediction)-np.array(test_lab))/len(prediction)
print(accuracy)

In [ ]:
aerror=np.count_nonzero(np.array(prediction)-np.array(test_lab))/len(prediction)
print(aerror)
accuracy=1 - aerror
print(accuracy)

In [ ]:
error=np.array(prediction)-np.array(test_lab)

import matplotlib.pyplot as plt
plt.figure(1)
plt.plot(error, linestyle="--")
# plt.legend(loc="best")
plt.xlabel('Records')
plt.ylabel('Error')
plt.show

# import csv
# with open('error.txt','w') as cfile:
#     w = csv.writer(cfile)
#     w.writerow(error)